# Importing Libraries and Loading Data

In [11]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

def loadData():
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip'
    response = requests.get(url)

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            with zip_ref.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                df = pd.read_csv(myfile, delim_whitespace=True, header=None)
            with zip_ref.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")

    return df, y


In [12]:
df, y = loadData()

labelEncoder = LabelEncoder()
yEncoded = labelEncoder.fit_transform(y.values.ravel())

/tmp/ipykernel_9121/3227644454.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
/tmp/ipykernel_9121/3227644454.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [13]:
standardizer = StandardScaler()
dfStandardized = standardizer.fit_transform(df)

# print(df[:20])

X_train_full, X_test_full, y_train, y_test = train_test_split(dfStandardized, yEncoded, test_size=0.2, random_state=42)

# Baseline model with all features using Naive Bayes

In [ ]:
startTime = time.time()
classifierPipelineComplete = Pipeline([
    ('classifier', GaussianNB())
])
classifierPipelineComplete.fit(X_train_full, y_train)
y_pred_full = classifierPipelineComplete.predict(X_test_full)
end_time = time.time()
full_features_time = end_time - start_time
accuracy_full = accuracy_score(y_test, y_pred_full)